# TME5 - t-SNE

## Imports

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pk

## Chargement des données

Les fonctions pour charger les bases Movie Lens 100k et Movie Lens 1M.
On récupère un dictionnaire pour les scores et un dictionnaire pour les dates.
Le première index de ces dictionnaires est l'identifiant de l'utilisateur, et le second les films notés.

In [2]:
def loadMovieLens(path='./data100k'):
    # Get movie titles
    movies={}
    for line in open(path+'/u.item'):
        (id,title)=line.split('|')[0:2]
        movies[id]=title
    # Load data
    prefs={} # Un dictionnaire User > Item > Rating
    times={} # Un dictionnaire User > Item > Timestamps
    for line in open(path+'/u.data'):
        (user,movieid,rating,ts)=line.split('\t')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
        times.setdefault(user,{})
        times[user][movies[movieid]]=float(ts)
    return prefs, times

In [3]:
def loadMovieLens1M(path='./data1m'):
    # Get movie titles
    movies={}
    for line in open(path+'/movies.dat'):
        id,title=line.split('::')[0:2]
        movies[id]=title
    # Load data
    prefs={}
    times={}
    for line in open(path+'/ratings.dat'):
        (user,movieid,rating,ts)=line.split('::')
        prefs.setdefault(user,{})
        prefs[user][movies[movieid]]=float(rating)
        times.setdefault(user,{})
        times[user][movies[movieid]]=float(ts)
    return prefs, times

## Représentations des données

Les matrices des scores Utilisateurs/Films sont des matrices de grandes dimensions mais sparses.
Afin de les manipuler efficacement, on emploiera 3 représentations différentes en même temps:
- Le dictionnaire des scores par utilisateurs: User > Item > Value
- Le dictionnaire des scores par films: Item > User > Value 
- La liste des triplets [User, Item, Value]

In [4]:
# Recupère une représentation des données sous la forme triplets [user, item, value] a partir d'un dictionnaire [User > item > value]
def getCouplesUsersItems(data):
    couples = []
    for u in data.keys():
        for i in data[u].keys():
            couples.append([u,i,data[u][i]])
    return couples

# Construit le dictionnaire des utilisateurs a partir des triplets [user, item, note]
def buildUsersDict(couples):
    dicUsers = {}
    for c in couples:
        if not c[0] in dicUsers.keys():
            dicUsers[c[0]] = {}
        dicUsers[c[0]][c[1]] = float(c[2])
    return dicUsers

# Construit le dictionnaire des objets a partir des triplets [user, item, note]
def buildItemsDict(couples):
    dicItems = {}
    for c in couples:
        if not c[1] in dicItems:
            dicItems[c[1]] = {}
        dicItems[c[1]][c[0]] = float(c[2])
    return dicItems

## Données de temps

Afin d'exploiter les données temporelles, on discrétise le temps en bins et on construit un vecteur qui a chaque triplets [user, item, value] associe le bins temporel correspondant.

In [5]:
def getTimeBins(couples, timedic, nbins):
    timestamps = np.zeros(len(couples))
    for i,c in enumerate(couples):
        timestamps[i] = timedic[c[0]][c[1]]
    time_bins = np.linspace(np.min(timestamps), np.max(timestamps), nbins+1)
    times = np.zeros(len(couples),int)
    for i in xrange(1,len(time_bins)):
        times = times + (timestamps > time_bins[i])
    return times

## Séparation des données en Train / Test

Pour pouvoir séparer les données en ensembles de Train et de Test, on utilisera la liste des triplets [User, Item, Scores].

In [6]:
# Split l'ensemble des triplets [user, item, note] en testProp% données de test et (1 - testProp) données de train
def splitTrainTest(couples,testProp):
    perm = np.random.permutation(couples)
    splitIndex = int(testProp * len(couples))
    return perm[splitIndex:], perm[:splitIndex]

# Modèles

On implémente ici les différents modèles. Les baselines prédisent simplement la note moyenne pour un utilisateur (ou pour un film) donné. Les modèles de factorisation matricielles tentent d'approximer les valeurs connues de la matrice des scores par un produit de deux matrices de dimensions inférieurs.

## Factorisation matricielle sans biais

On calcule les deux matrices P et Q tel que pour les exemples connus, PQ ~= X, où X est la matrice des scores.
Pour prédire, il suffit alors de lire dans la matrice PQ les nouveaux exemples.

In [7]:
class matrixFactorisation():
    def __init__(self, k, lambd=0.2, eps=1e-5, maxIter=2000, alternate=0):
        self.k = k
        self.lambd = lambd
        self.eps = eps
        self.maxIter = maxIter
        self.alternate = alternate #Pour l'optimisation alternée: 0 si non.
    def fit(self, dataUsers, dataItems, couples):
        self.p = {}
        self.q = {}
        self.loss = []
        #Choix du paramètre a optimisé en cas d'optimisation alternée
        optimP = True
        optimQ = (self.alternate == 0)
        for i in xrange(self.maxIter):
            loss = 0
            for j in xrange(len(couples)):
                #choix d'une entrée aléatoire
                r = np.random.randint(len(couples)) 
                user = couples[r][0]
                item = couples[r][1]
                # initialisation des nouveaux vecteurs p et q
                if not user in self.p:
                    self.p[user] = np.random.rand(1,self.k)
                if not item in self.q:
                    self.q[item] = np.random.rand(self.k,1)
                # Descente de gradient
                tmp = dataUsers[user][item] - self.p[user].dot(self.q[item])[0][0]
                if (optimP):
                    self.p[user] = (1 - self.lambd * self.eps) * self.p[user] + self.eps * 2 * tmp * self.q[item].transpose()
                if (optimQ):
                    self.q[item] = (1 - self.lambd * self.eps) * self.q[item] + self.eps * 2 * tmp * self.p[user].transpose()
                loss = loss + tmp*tmp #(Sans le terme de régularisation)
            self.loss.append(loss)
            # Optimisation alternée
            if (self.alternate != 0):
                if (i % self.alternate == 0):
                    optimP = optimQ
                    optimQ = 1 - optimQ
                    print i, loss / len(couples)
            else:
                if (i % 100 == 0):
                    print i, loss / len(couples)
    def predict(self, couplesTest):
        pred = np.zeros(len(couplesTest))
        for ind,c in enumerate(couplesTest):
            pred[ind] = self.p[c[0]].dot(self.q[c[1]])[0][0]
        return pred

# Tests les données Movie Lens 100k

Les données Movie Lens 100k comprennent 100 000 scores données par 1000 utilisateurs sur 1700 films.

## Préparation des données

On extrait aléatoirement une portion (20%) des données pour constituer la base de test, et le reste sera utilisé en apprentissage.

Comme on ne souhaite ne pas évaluer les objets et les utilisateurs qui n'ont jamais été rencontré en apprentissage, on retire les couples correspondants de l'ensemble de test.

Reste ensuite à reconstruire les deux dictionnaires a partir de ces liste de couples.

In [8]:
# Chargement
data, timestamps = loadMovieLens()

# Récupérer la représentation en liste de triplets
couples = getCouplesUsersItems(data)

# La séparer en ensemble d'apprentissage et de test
trainCouples, testCouples = splitTrainTest(couples,.20)

# Reconstruire les dictionnaires pour l'ensemble d'apprentissage
trainUsers = buildUsersDict(trainCouples)
trainItems = buildItemsDict(trainCouples)

# Supprimer de l'ensemble de test les éléments inconnus en apprentissage
toDel = []
for i,c in enumerate(testCouples):
    if not c[0] in trainUsers:
        toDel.append(i)
    elif not c[1] in trainItems:
        toDel.append(i)
testCouples = np.delete(testCouples, toDel, 0)

# Reconstruire les dictionnaires pour l'ensemble de test
testUsers  = buildUsersDict(testCouples)
testItems  = buildItemsDict(testCouples)

# taille des données
#print len(trainUsers), len(testUsers)
#print len(trainItems), len(testItems)

## Factorisation matricielle sans biais

Après apprentissage, la factorisation matricielle donne une erreur moyenne en test de 0.9
Il est meilleur que les baselines de 0.1 point.
On note que le loss en apprentissage est de 0.84. 
Il est possible que l'on obtienne de meilleurs score de généralisation en test en augmentant la régularisation mais au vu du score actuel en apprentissage, le gain devrait rester assez faible.

In [10]:
model = matrixFactorisation(10, alternate=0)
model.fit(trainUsers, trainItems, trainCouples)

0 2.71825447884
100 1.30461910265
200 1.07198937601
300 0.988878955293
400 0.947013141787
500 0.930374148402
600 0.901617765719
700 0.890678619669
800 0.88196674407
900 0.867942896422
1000 0.863288863963
1100 0.86357578657
1200 0.854815106275
1300 0.847335834343
1400 0.847021165112
1500 0.846547840724
1600 0.854972726777
1700 0.845742778706
1800 0.849065445756
1900 0.831846368653


In [11]:
pk.dump(model,  open("model.p", "wb"))

In [12]:
plt.figure()
plt.plot(model.loss)
plt.show()

In [17]:
pred = model.predict(testCouples)
print "Erreur de test:", ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

Erreur de test: 0.909580818424


# Experiences sur les données Movie Lens 1M

Le dataset Movie Lens 1M contient 1 million d'entrées, données par 6000 utiilisateurs sur 4000 films.

Une remarque que l'on peut déjà faire est que la matrice des scores est "moins sparse" que celle issue de la base 100k.

En effet, pour la base 100k on a 100000 notes pour une matrice 1000x1700, soit un remplissage de 17% de la matrice.

Pour la base 1M, on a 1 000 000 de notes pour une matrice 6000x4000, soit 24% de remplissage.

## Préparation des données

In [24]:
# Chargement
data, timestamps = loadMovieLens1M()

# Récupérer la représentation en liste de triplets
couples = getCouplesUsersItems(data)

# Séparer en ensemble d'apprentissage et de test
trainCouples, testCouples = splitTrainTest(couples,.20)

# Reconstruire les dictionnaires pour l'ensemble d'apprentissage
trainUsers = buildUsersDict(trainCouples)
trainItems = buildItemsDict(trainCouples)

# Supprimer de l'ensemble de test les éléments inconnus en apprentissage
toDel = []
for i,c in enumerate(testCouples):
    if not c[0] in trainUsers:
        toDel.append(i)
    elif not c[1] in trainItems:
        toDel.append(i)
testCouples = np.delete(testCouples, toDel, 0)

# Reconstruire les dictionnaires pour l'ensemble de test
testUsers  = buildUsersDict(testCouples)
testItems  = buildItemsDict(testCouples)

# taille des données
#print len(trainUsers), len(testUsers)
#print len(trainItems), len(testItems)

## Factorisation Matricielle

Avec un score en apprentissage de 0.82 et de généralisation de 0.85, on peut estimer que le paramètre de régularisation choisi (lambda = 0.2) est raisonnable pour ce problème. La factorisation matricielle est aussi meilleure que celle de la base 100k, probablement parceque la matrice d'apprentissage est moins sparse.

In [27]:
model8 = matrixFactorisation(10, alternate=0, maxIter=1000)
model8.fit(trainUsers, trainItems, trainCouples)

0 2.85630500372
100 0.989208053185
200 0.894268607039
300 0.859435186046
400 0.843744226415
500 0.834776300235
600 0.829587454159
700 0.826814746078
800 0.820488518828
900 0.820793261528


In [28]:
plt.figure()
plt.plot(model8.loss)
plt.show()

In [29]:
pred = model8.predict(testCouples)
print ((pred - np.array(testCouples[:,2], float)) ** 2).mean()

0.848198539785


# Conclusion

Dans ce travail, nous avons implémenté des modèles de filtrage collaboratif que nous avons ensuite chercher à évaluer.
Si on a observé qu'un modèle simple de factorisation matricielle pouvait obtenir des résultats concluants, meilleurs qu'une baseline naïve, nous n'avons pu démontré d'intérêt pratiques des modèles avec biais.

Cependant, nous avons observés que si nos modèles sans biais avaient des scores similaires en apprentissage qu'en généralisation, ce n'était pas le cas de nos modèles avec biais qui obtiennent de bien meilleurs scores en apprentissage qui ne se traduisent pas en test. On peut en déduire que nous n'avons pas déterminé les bons hyperparamètres pour permettre une bonne généralisation, et que vraisemblablement on peut encore augmenter le score en généralisation de nos modèles avec biais.

Enfin, nous n'avons eu le temps ni d'évaluer les modèles avec biais temporel, ni l'influence de la dimension de factorisation matricielle, que l'on peut aussi voir comme une forme de régularisation.

In [70]:
A = np.array([[1,2,3],[4,5,6],[1,3,2],[4,5,6]])
print A

[[1 2 3]
 [4 5 6]
 [1 3 2]
 [4 5 6]]


In [71]:
norm = np.sum(A**2,1)
normi = np.reshape(norm, (1, np.shape(norm)[0]))
normi = np.repeat(normi, np.shape(A)[0], 0)
distance = normi.T + normi - 2 * A.dot(A.T)
print distance

[[ 0 27  2 27]
 [27  0 29  0]
 [ 2 29  0 29]
 [27  0 29  0]]


In [75]:
p = np.exp(- distance / (2 * np.array([2.,3.,4.,5.]))) #sigma²
print p

[[ 1.          0.011109    0.77880078  0.06720551]
 [ 0.00117088  1.          0.0266491   1.        ]
 [ 0.60653066  0.00795994  1.          0.05502322]
 [ 0.00117088  1.          0.0266491   1.        ]]


In [82]:
np.sum(p - np.eye(np.shape(p)[0]),0)

array([ 0.60887242,  1.01906894,  0.83209898,  1.12222873])

In [84]:
ps=p / np.sum(p - np.eye(np.shape(p)[0]),0)
print ps

[[ 1.64238019  0.01090112  0.93594729  0.05988575]
 [ 0.00192303  0.98128788  0.03202636  0.89108394]
 [ 0.99615394  0.007811    1.20178011  0.04903031]
 [ 0.00192303  0.98128788  0.03202636  0.89108394]]


In [86]:
(ps + ps.T) / (2*np.shape(ps)[0])

array([[ 0.41059505,  0.00160302,  0.24151265,  0.0077261 ],
       [ 0.00160302,  0.24532197,  0.00497967,  0.23404648],
       [ 0.24151265,  0.00497967,  0.30044503,  0.01013208],
       [ 0.0077261 ,  0.23404648,  0.01013208,  0.22277098]])

In [108]:
class tSNE():
    def __init__(self, perp, nIter, lr, moment, dim=2):
        self.perp = perp # entre 5 et 50
        self.nIter = nIter
        self.lr = lr
        self.moment = moment
        self.dim = dim
    def fit(self, data):
        #--------------------------------------------------------------------------#
        #initialiser y et q(j|i)
        self.embedding = np.random.randn(np.shape(data)[0], self.dim) * 1e-4
        self.qjsi = np.zeros((self.dim,self.dim))
        #--------------------------------------------------------------------------#
        #initialisation des sigmas et des p(j|i)
        self.pjsi = np.zeros((np.shape(data)[0],np.shape(data)[0]))
        self.sigmai = np.zeros(np.shape(data)[0]) #il faut calculer les sigma²
        self.perpi = np.zeros(np.shape(data)[0])
        #initialisation bornes
        normx = np.sum((data**2),1)
        repnormx = np.reshape(normx, (1, np.shape(normx)[0]))
        distancex = repnormx + repnormx.T - 2 * data.dot(data.T)
        self.sigmaisup = np.ones(np.shape(data)[0]) * distancex.max()
        self.sigmaiinf = np.zeros(np.shape(data)[0])
        print(self.sigmaisup.shape)
        print(self.sigmaiinf.shape)
        self.sigmai = (self.sigmaisup + self.sigmaiinf) / 2.
        stop = 0
        while stop < 10:
            # Matrice des distances ||xi - xj||²
            normx = np.sum((data**2),1)
            repnormx = np.reshape(normx, (1, np.shape(normx)[0]))
            distancex = repnormx + repnormx.T - 2 * data.dot(data.T)
            # p(j|i) #en ligne les j, en colonne les i
            self.pjsi = np.exp( -distancex / (2 * (self.sigmai**2).reshape((1, self.sigmai.shape[0])) )) 
            self.pjsi = self.pjsi / np.sum(self.pjsi - np.eye(self.pjsi.shape[0]), 0)
            self.perpi = np.power(2, -np.sum(self.pjsi * np.log2(self.pjsi), 1))
            print(self.sigmai)
            print(self.perpi)
            
            self.sigmai = self.sigmaisup
            self.pjsi = np.exp( -distancex / (2 * (self.sigmai**2).reshape((1, self.sigmai.shape[0])) )) 
            self.pjsi = self.pjsi / np.sum(self.pjsi - np.eye(self.pjsi.shape[0]),0)
            self.perpi = np.power(2, -np.sum(self.pjsi * np.log2(self.pjsi), 1))
            print(self.sigmai)
            print(self.perpi)
            break
            
            self.difPerp = self.perpi - self.perp
            if np.sum( (self.difPerp * np.sign(self.difPerp)) < 1e-2 ) > 0:
                break
            else:
                self.sigmaisup[self.difPerp > 0] = self.sigmai[self.difPerp > 0]
                self.sigmaiinf[self.difPerp < 0] = self.sigmai[self.difPerp < 0]
                self.sigmai = (self.sigmaisup + self.sigmaiinf) / 2
                stop += 1
                print('-------------')
                print('sigmaisup') 
                print(self.sigmaisup)
                print('sigmaiinf') 
                print(self.sigmaiinf)
                print('sigmai') 
                print(self.sigmai)
                print('perpi') 
                print(self.perpi)
                print('difPerp')
                print(self.difPerp)

In [109]:
model = tSNE(perp=5, nIter=100, lr=1e-3, moment=0.4, dim=2)
data = np.random.rand(5,10)
model.fit(data)

(5,)
(5,)
[ 0.88673163  0.88673163  0.88673163  0.88673163  0.88673163]
[ 4.91108741  5.27588474  5.41632133  6.01848571  5.61279523]
[ 1.77346327  1.77346327  1.77346327  1.77346327  1.77346327]
[ 5.65988018  5.70946989  5.73461723  5.83950589  5.76283878]


In [45]:
class tSNE():
    def __init__(perp, nIter, lr, moment, dim=2):
        self.perp = perp # entre 5 et 50
        self.nIter = nIter
        self.lr = lr
        self.moment = moment
        self.dim = dim
    def fit(data):
        #--------------------------------------------------------------------------#
        #initialiser y et q(j|i)
        self.embedding = np.random.randn(np.shape(data)[0], self.dim) * 1e-4
        self.qjsi = np.zeros((self.dim,self.dim))
        #--------------------------------------------------------------------------#
        #initialisation des sigmas et des p(j|i)
        self.pjsi = np.zeros((np.shape(data)[0],np.shape(data)[0]))
        self.sigmai = np.zeros(np.shape(data)) #il faut calculer les sigma²
        self.perpi = np.zeros(np.shape(data))
        #initialisation bornes
        normx = np.sum((data**2),1)
        repnormx = np.reshape(normx, (1, np.shape(normx)[0]))
        distancex = repnormx + repnormx.T - 2 * data.dot(data.T)
        self.sigmaisup = np.ones(np.shape(data)) * distancex.max()
        self.sigmaiinf = np.zeros(np.shape(data)
        self.sigmai = sigmaisup + sigmaiinf / 2
        while True:
            # Matrice des distances ||xi - xj||²
            normx = np.sum((data**2),1)
            repnormx = np.reshape(normx, (1, np.shape(normx)[0]))
            distancex = repnormx + repnormx.T - 2 * data.dot(data.T)
            # p(j|i) #en ligne les j, en colonne les i
            self.pjsi = np.exp(-distancex / 2 * self.sigmai ) 
            self.pjsi = self.pjsi / np.sum(self.pjsi - np.eye(np.shape(self.pjsi)[0],0))
            self.perpi = np.power(2, -1 * np.sum(self.pjsi * np.log2(self.pjsi), 0))
            difPerp = self.perpi - self.perp
            if np.sum( (difPerp * np.sign(difPerp)) < 1e-2 ) == 0:
                break
            else:
                self.sigmaisup[difPerp < 0] = self.sigmai
                self.sigmaiinf[difPerp > 0] = self.sigmai
                self.sigmai = sigamisup + sigmaiinf / 2
        
        # p(ij)
        self.pij = np(self.pjsi + self.pjsi.T) / (2*np.shape(self.pjsi)[0])
                                  
        # Descente de Gradient
        for t in xrange(self.nIter):
            # Matrice des distances ||yi - yj||²
            normy = np.sum((self.embedding**2),1)
            repnormy = np.reshape(normy, (1, np.shape(normy)[0]))
            distancey = repnormy + repnormy.T - 2 * self.embedding.dot(self.embedding.T)
            # q(ij)
            self.qij = 1 + distancey
            self.qij = np.sum(self.qij - np.eye(np.shape(self.qij)[0],0)) / self.qij
            # Gradient
            tmpgrad = 4 * ((self.pij - self.qij) / (1 + distancey))
            
        
        

        pass

SyntaxError: invalid syntax (<ipython-input-45-5db118d56fc2>, line 17)

In [89]:
A

array([[1, 2, 3],
       [4, 5, 6],
       [1, 3, 2],
       [4, 5, 6]])

In [91]:
A[0] - A

array([[ 0,  0,  0],
       [-3, -3, -3],
       [ 0, -1,  1],
       [-3, -3, -3]])

In [94]:
np.sum(A[0] - A, 0)

array([-6, -7, -5])

In [95]:
B = np.array([[0,1],[1,2],[3,4],[4,1]])
print B

[[0 1]
 [1 2]
 [3 4]
 [4 1]]


In [ ]:
M = np.zeros(( ,2))